We begin with an Excel workbook that contains a very messy structure of data. It contains:
-Summary, where each transmission lines analysis results can be viewed one-by-one by an end user
-Impacts, where the impact of queue projects are evaluated on each transmission line
-POI List, which lists each Point of Interconnection evaluated
-Queue List, which lists each queue project evaluated
-Table Summary, which contains a table for each state with a hierarchy that shows the charge and inject capacity across lines as they are affected by different queue projects.

We also have a KML file that stores HIFLD data for each of these lines. We are interested in this data because it includes geographic features which we can map our data analysis to.

Use Case
With brevity in mind, I will conduct the analysis on a set of 3 states, which include Georgia, Alabama, and Mississipi

I will begin by making the table summaries each their own page of the workbook and deleting ones we will not use. There is little enough data that this is feasible.


USE CASE

We want to take the study done by an electrical engineering consultant and parse it to identify which transmission lines are viable for new renewable energy projects. We then want to turn these transmission lines into a map, store the data we synthesize during our analysis, and do some geographic analyses on them. We define our non-geographic viability criteria as a dictionary of lists. The dictionary's lists will be filled with values which represent arbitrary breaks in our data that we will use for comparison and analysis. 

While we start with the entire workbook, I know from browsing the tables that there are 2 in particular that we are interested in for our analyses. These are the "POI List" which contains all of the details about our transmission lines, and "Impacts", which contains the data we will use to determine viability. Let's take a look:

In [16]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
import numpy as np
import fiona
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'


In [17]:
ImpactsDF = pd.read_excel(r"C:\Users\bobro\Personal_Projects\Pterra Study - Copy.xlsx", 1)
display(ImpactsDF)

,ISO,Owner,POI ID,POI Description,Voltage Level,Remarks,MW Capacity without QP,Largest Negative MW Impact,QPs with Significant Impacts (at least 30 MW reduction or caused the largest impact)
0,PJM,CE,IL-1,Belvidere - Marengo,138.0,NaN,170.0,-170.0,"AH2-197, AG2-545, AG2-544, AF1-048"
1,PJM,CE,IL-2,Marengo - Woodstock,138.0,NaN,130.0,-130.0,"AH2-197, AG2-545, AG2-544"
2,PJM,CE,IL-3,Cherry Valley - DeKalb,138.0,NaN,80.0,-40.0,"AH2-234, AG2-575, AG1-119"
3,PJM,CE,IL-4,Cherry Valley - Silver Lake,345.0,NaN,390.0,-280.0,"AH2-234, AH2-197, AH2-085, AH1-657, AH1-047, A..."
4,PJM,CE,IL-5,Crescent Ridge - La Salle,138.0,NaN,150.0,-150.0,"AH2-372, AG1-005, AF1-318"
...,...,...,...,...,...,...,...,...,...
211,PJM,AEP,MI-2,New Buffalo - Three Oaks,69.0,NaN,30.0,0.0,NaN
212,PJM,AEP,MI-3,New Carlisle - Hickory Creek,138.0,NaN,150.0,-150.0,AH2-373
213,PJM,AEP,MI-4,Bridgman - Pletcher,69.0,NaN,60.0,-10.0,AH2-418
214,PJM,AEP,MI-5,Pletcher - Buchanan South Side,69.0,NaN,70.0,-70.0,AH2-373


Here we can see some really interesting data. We have:
    POI ID
    POI Description
    Voltage Level
    MW Cap. without QP
    Largest Negative MW Impact

These will be the core of our analysis. We are interested only in economically feasible lines, which means there are criteria for MW capacity based on the Voltage level.
We also want to avoid investing resources into transmission lines where competition projects already in the queue could fill up all of the available capacity on a transmission line. The risk is that the project requires a transmission upgrade, which is an economic disaster and would likely result in an abandoned project.

For simplicity's sake, I take the largest negative MW impact as the worst case scenario. There are other possibilities that may be better or worse, but the probabilities of those scenarios are difficult to calculate and are beyond the scope of our data. Additionally, the real numbers behind the original analyses are proprietary, so I will be using arbitrary criteria as follows:

For eligibility, the largest negative MW impact on a POI may not reduce that POIs capacity below:

    >=69 kV    ----> 80 MW
    >=115 kV   ----> 100 MW
    >=138 kV   ----> 120 MW
    >=161 kV   ----> 140 MW
    >=230 kV   ----> 160 MW
    >=345 kV   ----> 180 MW

Let's see which lines make the cut:
 




In [18]:
ImpactsDF['Eval_Capacity'] = ImpactsDF['MW Capacity without QP'] + ImpactsDF['Largest Negative MW Impact']
ImpactsDF['viable'] = pd.Series
ImpactsDF['viable'] = np.where((ImpactsDF['Voltage Level'] <= 69) & (ImpactsDF['Eval_Capacity'] >= 80), True, ImpactsDF['viable'])
ImpactsDF['viable'] = np.where(((ImpactsDF['Voltage Level'] == 115) & (ImpactsDF['Eval_Capacity'] >= 100)), True, ImpactsDF['viable'])
ImpactsDF['viable'] = np.where(((ImpactsDF['Voltage Level'] == 138) & (ImpactsDF['Eval_Capacity'] >= 120)), True, ImpactsDF['viable'])
ImpactsDF['viable'] = np.where(((ImpactsDF['Voltage Level'] == 161) & (ImpactsDF['Eval_Capacity'] >= 140)), True, ImpactsDF['viable'])
ImpactsDF['viable'] = np.where(((ImpactsDF['Voltage Level'] == 230) & (ImpactsDF['Eval_Capacity'] >= 160)), True, ImpactsDF['viable'])
ImpactsDF['viable'] = np.where(((ImpactsDF['Voltage Level'] == 345) & (ImpactsDF['Eval_Capacity'] >= 180)), True, ImpactsDF['viable'])

ImpactsDF.drop(ImpactsDF[ImpactsDF.viable != True].index, inplace=True)

display(ImpactsDF[['POI ID', 'Voltage Level', 'MW Capacity without QP', 'Largest Negative MW Impact', 'Eval_Capacity', 'viable']])


,POI ID,Voltage Level,MW Capacity without QP,Largest Negative MW Impact,Eval_Capacity,viable
7,IL-8A,138.0,230.0,-90.0,140.0,True
11,IL-10,138.0,240.0,-120.0,120.0,True
12,OH-1A,138.0,390.0,0.0,390.0,True
13,OH-1B,138.0,240.0,-10.0,230.0,True
14,OH-2,138.0,210.0,-70.0,140.0,True
15,OH-3,345.0,600.0,-200.0,400.0,True
19,OH-6A,138.0,150.0,-20.0,130.0,True
20,OH-6B,138.0,200.0,-50.0,150.0,True
22,OH-8,138.0,190.0,-20.0,170.0,True
23,OH-9,138.0,230.0,-100.0,130.0,True


Great! It looks like the calculations and evaluations are working right. We went from 216 rows down to 52 after dropping lines that aren't viable, and we can now take these and match them to transmission lines from HIFLDs data to generate mappable linestrings. Lets take a look at our KML.

In [19]:
KML_GDF = gpd.GeoDataFrame()
for layer in fiona.listlayers(r"C:\Users\bobro\OneDrive\Documents\Transmission Analysis Map Data.kml"):
    layer_gdf = gpd.read_file(r"C:\Users\bobro\OneDrive\Documents\Transmission Analysis Map Data.kml", driver='KML', layer=layer)
    KML_GDF = pd.concat([KML_GDF, layer_gdf], ignore_index=True)
display(KML_GDF)



,Name,Description,geometry
0,Claxton - Highway 301,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-81.85080 32.29540 40.0000...
1,Forstmann Louisville - North Louisville Tap,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-82.50060 32.91560 40.0000...
2,Donalsonville - Reynoldsville,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-84.88380 31.04300 40.0000...
3,Blakely - Pine Hill Junction,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-84.90600 31.37160 40.0000...
4,North Glennville - Stoney Hill,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-82.03150 31.98830 40.0000...
...,...,...,...
559,Three Oaks,,POINT Z (-86.60633 41.80048 0.00000)
560,NEW BUFFALO,,POINT Z (-86.73828 41.79021 0.00000)
561,COOK,,POINT Z (-86.55417 41.97679 0.00000)
562,KENZIE CREEK,,POINT Z (-86.18955 41.88600 0.00000)


Awesome! It looks like "Name" attributes from the KML match up with "POI Description" from our Impacts Dataframe. Let's do a Join on that column


In [20]:
Viable_Lines_GDF = gpd.GeoDataFrame(ImpactsDF.merge(KML_GDF, how='inner', left_on='POI Description', right_on='Name'))
display(Viable_Lines_GDF)

,ISO,Owner,POI ID,POI Description,Voltage Level,Remarks,MW Capacity without QP,Largest Negative MW Impact,QPs with Significant Impacts (at least 30 MW reduction or caused the largest impact),Eval_Capacity,viable,Name,Description,geometry
0,SE,SOCO,GA-1,Claxton - Highway 301,115.0,NaN,250.0,-20.0,IC-625,230.0,True,Claxton - Highway 301,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-81.85080 32.29540 40.0000...
1,SE,SOCO,GA-5,North Glennville - Stoney Hill,115.0,NaN,100.0,0.0,NaN,100.0,True,North Glennville - Stoney Hill,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-82.03150 31.98830 40.0000...
2,SE,SOCO,GA-11,Bio - Vanna,230.0,NaN,350.0,0.0,NaN,350.0,True,Bio - Vanna,Transmission Line - 230kV - GEORGIA POWER CO,MULTILINESTRING Z ((-82.95470 34.29540 40.0000...
3,SE,SOCO,GA-12,North Tifton - Mithchell 230 Ring Bus,230.0,NaN,310.0,-130.0,"IC-1030, IC-1032, IC-575, IC-724, IC-774, IC-8...",180.0,True,North Tifton - Mithchell 230 Ring Bus,Transmission Line - 230kV - GEORGIA POWER CO,MULTILINESTRING Z ((-83.54880 31.47650 40.0000...
4,SE,SOCO,GA-22,Pitts - North Tifton,230.0,NaN,400.0,-190.0,"IC-641, IC-908, IC-944",210.0,True,Pitts - North Tifton,Transmission Line - 230kV - GEORGIA POWER CO,MULTILINESTRING Z ((-83.54880 31.47640 40.0000...
5,SE,SOCO,GA-25,Pitts - Kathleen,115.0,NaN,250.0,-90.0,"IC-641, IC-908, IC-944",160.0,True,Pitts - Kathleen,Transmission Line - 230kV - GEORGIA POWER CO,MULTILINESTRING Z ((-83.56280 31.98660 40.0000...
6,SE,SOCO,GA-34,Claxton - North Glennville,115.0,NaN,160.0,-30.0,IC-671,130.0,True,Claxton - North Glennville,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-81.89960 32.17420 40.0000...
7,SE,SOCO,GA-35,Glenville - South Glennville,115.0,NaN,160.0,-10.0,IC-671,150.0,True,Glenville - South Glennville,Transmission Line - 115kV - GEORGIA POWER CO,MULTILINESTRING Z ((-81.91320 31.92990 40.0000...
8,SE,TVA,AL-1,Limestone - Mallard Fox Creek Tap,161.0,NaN,230.0,-30.0,385,200.0,True,Limestone - Mallard Fox Creek Tap,Transmission Line - 161kV - TENNESSEE VALLEY A...,MULTILINESTRING Z ((-86.84630 34.71890 40.0000...
9,SE,PS,AL-2,Chastain - Repton,115.0,NaN,140.0,0.0,NaN,140.0,True,Chastain - Repton,Transmission Line - 115kV - ALABAMA POWER CO,MULTILINESTRING Z ((-87.27740 31.42100 40.0000...


Viable_Lines_Errors = ImpactsDF.merge(KML_GDF, how='left', left_on='POI Description', right_on='Name')
ImpactsDF.drop(ImpactsDF[ImpactsDF.viable != True].index, inplace=True)

In [21]:
Viable_Lines_Errors = ImpactsDF.merge(KML_GDF, how='left', left_on='POI Description', right_on='Name', indicator='source')
Viable_Lines_Errors.drop(Viable_Lines_Errors[Viable_Lines_Errors.geometry != None].index, inplace=True)
display(Viable_Lines_Errors)

,ISO,Owner,POI ID,POI Description,Voltage Level,Remarks,MW Capacity without QP,Largest Negative MW Impact,QPs with Significant Impacts (at least 30 MW reduction or caused the largest impact),Eval_Capacity,viable,Name,Description,geometry,source
0,PJM,CE,IL-8A,Rock Falls - Kewanee,138.0,NaN,230.0,-90.0,"AH2-376, AH2-375, AH2-204, AH2-015, AG1-005, A...",140.0,True,NaN,NaN,None,left_only
1,PJM,CE,IL-10,Steward - Waterman,138.0,NaN,240.0,-120.0,"AH2-234, AG2-575, AG1-119, AD1-098",120.0,True,NaN,NaN,None,left_only
2,PJM,AEP,OH-1A,South East Canton - Blue Bell,138.0,NaN,390.0,0.0,NaN,390.0,True,NaN,NaN,None,left_only
3,PJM,AEP,OH-1B,South East Canton - Blue Bell,138.0,NaN,240.0,-10.0,"AH1-655, AH1-350, AG2-031",230.0,True,NaN,NaN,None,left_only
4,PJM,ATSI,OH-2,Toussaint - Ottawa,138.0,NaN,210.0,-70.0,AE2-176,140.0,True,NaN,NaN,None,left_only
5,PJM,ATSI,OH-3,Davis Besse - Beaver,345.0,NaN,600.0,-200.0,"AH2-345, AH2-156, AH1-425, AH1-095, AH1-094, A...",400.0,True,NaN,NaN,None,left_only
6,PJM,ATSI,OH-6A,Galion - Dual Rail Tap,138.0,NaN,150.0,-20.0,AH1-521,130.0,True,NaN,NaN,None,left_only
7,PJM,ATSI,OH-6B,Galion - Dual Rail Tap,138.0,NaN,200.0,-50.0,AG2-119,150.0,True,NaN,NaN,None,left_only
8,PJM,ATSI,OH-8,East Springfield - Millcreek,138.0,NaN,190.0,-20.0,AE2-217,170.0,True,NaN,NaN,None,left_only
9,PJM,ATSI,OH-9,East Springfield - London Tap,138.0,NaN,230.0,-100.0,"AH1-319, AE2-217, AC1-078",130.0,True,NaN,NaN,None,left_only


Excellent! I'll review these ones and match up the best ones with lines on the KML using their locations and descriptions.

For now, let's take a look at the map we've got so far:

In [22]:
Viable_Lines_GDF = Viable_Lines_GDF.explode()

map = KeplerGl()
map.add_data(data=Viable_Lines_GDF, name='Viable Transmission Lines')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\bobro\anaconda3\envs\Transmission_Analysis\lib\site-packages\jupyter_client\session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(data={'Viable Transmission Lines': {'index': [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0),…